In [5]:
from sklearn.neighbors.unsupervised import NearestNeighbors

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from joblib import dump, load

Nous avons les sommes des activité telco des utilisateur sur les 6 dérnier mois. Les variables sont catégorisé en deux groupes. Il y a les variables originaux  (age, mosen , mosen), et les variables calculés (cavoice_catrechrg = cavoice / catrechg). Dans cette analyse nous nous intéresseront aux variations des balance des clients orange money


In [7]:
import pandas as pd
import warnings
import seaborn as sns
from sklearn.decomposition import PCA 
from sklearn.preprocessing import Normalizer
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
warnings.filterwarnings('ignore')

In [8]:
#sql = 'SELECT * FROM `pamf-dwh.DFS_VIEW.approved`'
sql = '''SELECT subs.customer_id, cust.orange_id ,cust.alias  FROM `pamf-dwh.PAMF_AREA.fact_subscription` as subs 
join `pamf-dwh.PAMF_AREA.dim_customer` as cust on cust.customer_id = subs.customer_id 
where status = 1 and product_id = 1'''
credentials_path = '../pamf-dwh-775beaae50ff.json'
project_id = 'pamf-dwh'
approved = pd.read_gbq( sql, project_id, private_key=credentials_path, dialect='standard')

In [22]:
approved

,customer_id,orange_id,alias
0,190217,PT131226.0938.D00638,PCPAMFMGMG00000190217
1,190258,PT190903.1016.525825,PCPAMFMGMG00000190258
2,190290,PT190209.0918.487068,PCPAMFMGMG00000190290
3,190315,PT180622.1157.812988,PCPAMFMGMG00000190315
4,190328,PT190605.0955.565429,PCPAMFMGMG00000190328
5,190338,PT120502.1030.000162,PCPAMFMGMG00000190338
6,190389,PT180206.1126.600520,PCPAMFMGMG00000190389
7,190419,PT171011.1559.090835,PCPAMFMGMG00000190419
8,190222,PT170318.0843.660391,PCPAMFMGMG00000190222
9,190233,PT190221.0952.555120,PCPAMFMGMG00000190233


In [9]:
dd = pd.read_csv('./data_OrangeNov2019_remaining.csv',sep=',')
def renameCols (collist):
    dnewcol={}
    for c in collist:
        newname = c.split(';')[0].replace(' ','')
        dnewcol[c]=newname
    return dnewcol
dd.rename(columns=renameCols(dd.columns),inplace=True)

In [10]:
neiboursSearch = load('15NN_sept_similarity.pkl')

In [11]:
dd.head(2)

,Unnamed:0,usr_id,age,cin,omsen,mosen,nr_rechrg,nr_erechrg,ca_rechrg,ca_erechrg,...,tcf_catcashin,cat_cashout,nrt_cashin,t_inout,nrt_cashout,ca_sms,type,Alias,OrangeId,ID_Number
0,0,PT180920.0731.821065,24.0,711412004849,408,408,0,0,0,0,...,0.0000,2000,1,2,1,0,prepaid,PCPAMFMGMG00000150451,PT180920.0731.821065,7.11412e+11
1,1,PT160509.1047.107626,29.0,715032021438,1272,4128,21,98,22000,73500,...,-0.0773,400000,4,9,5,120,prepaid,NaN,NaN,NaN


In [12]:
walletCol = ['cat_cashin' ,'cat_cashout','t_cf' ,'nrt_cashin'  ,'nrt_cashout' ,'t_inout' ,"nr_p2pin",'nr_p2pout','nr_saly' ,'ca_p2pin','ca_p2pout','ca_saly','ca_cashout','ca_cashin']
data = dd[walletCol+['usr_id','cin',]]
#karama = pd.read_excel('./Liste_beneficiaires_wallet_karama_actifs_01_au_31_juillet-19.xlsx')
#data= karama[['USER_ID']].merge(data,left_on='USER_ID',right_on='usr_id',how='right') # .describe()
#data.rename(columns={'USER_ID':'indice'},inplace=True)
#data['indice'] = data['indice'].apply(lambda x: 1 if x is not np.nan else 0)
#data['is_saly'] = data.ca_saly.apply(lambda x: 1 if x >0 else 0)

In [13]:
#initialBase = './bestBigloaner.xlsx'

initialBase = './bestBigsaver.xlsx'

In [14]:
data.head(2)# .usr_id

,cat_cashin,cat_cashout,t_cf,nrt_cashin,nrt_cashout,t_inout,nr_p2pin,nr_p2pout,nr_saly,ca_p2pin,ca_p2pout,ca_saly,ca_cashout,ca_cashin,usr_id,cin
0,2000,2000,0,1,1,2,0,0,0,0,0,0,2000,2000,PT180920.0731.821065,711412004849
1,371300,400000,-28700,4,5,9,0,0,0,0,0,0,400000,371300,PT160509.1047.107626,715032021438


In [15]:
def readMaxMoneyVal(filename):
    ''' this function return the PT of our savers '''
    df_money = pd.read_excel(filename)
    #sql = ''' SELECT rContactID, OrangeId FROM `pamf-dwh.DFS_VIEW.approved`  '''
    sql = '''SELECT cast(subs.customer_id as int64) as rContactID, cust.orange_id ,cust.alias  FROM `pamf-dwh.PAMF_AREA.fact_subscription` as subs 
join `pamf-dwh.PAMF_AREA.dim_customer` as cust on cust.customer_id = subs.customer_id 
where status = 1 and product_id = 1'''
    credentials_path = '../pamf-dwh-775beaae50ff.json'
    project_id = 'pamf-dwh'
    registered = pd.read_gbq( sql, project_id, private_key=credentials_path, dialect='standard')
    #registered.rContactID = registered.rContactID.apply(lambda x:int(x))
    #df_money.rContactID = df_money.rContactID.apply(lambda x:int(x))
    #registered.rename(columns={'customer_id':'rContactID'},inplace=True)
    #return df_money.merge(registered,on='rContacID', how='inner')[['orange_id']]
    return df_money,registered

In [16]:
df_money, registered = readMaxMoneyVal(initialBase)

In [17]:
df_money.rContactID = df_money.rContactID.apply(lambda x:int(x))
registered.rContactID = registered.rContactID.apply(lambda x:int(x))

In [18]:
df_money = df_money.merge(registered, on='rContactID')[['orange_id']] #.head(2)

In [19]:
"""def readMaxMoneyVal(filename):
    ''' this function return the PT of our savers '''
    df_money = pd.read_excel(filename)
    #sql = ''' SELECT rContactID, OrangeId FROM `pamf-dwh.DFS_VIEW.approved`  '''
    sql = '''SELECT subs.customer_id, cust.orange_id ,cust.alias  FROM `pamf-dwh.PAMF_AREA.fact_subscription` as subs 
join `pamf-dwh.PAMF_AREA.dim_customer` as cust on cust.customer_id = subs.customer_id 
where status = 1 and product_id = 1'''
    credentials_path = '../pamf-dwh-775beaae50ff.json'
    project_id = 'pamf-dwh'
    registered = pd.read_gbq( sql, project_id, private_key=credentials_path, dialect='standard')
    registered.rename(columns={'customer_id':'rContactID'},inplace=True)
    return df_money.merge(registered,on='rContacID')[['orange_id']]"""

'''data = data.merge(readMaxMoneyVal(initialBase), left_on='usr_id',right_on='orange_id',how='left')
'''
data = data.merge(df_money, left_on='usr_id',right_on='orange_id', how='left')


In [20]:
data['is_good_saver'] = data['orange_id'].apply(lambda x: 1 if x is not np.nan else 0)
#del data['OrangeId']


''''''

''

In [21]:
#data['is_one'] = data.is_good_saver  |  data.is_saly | data.indice

 #[data['orange_id']=='PT100702.0619.000001']

In [22]:
useableCol = ['cat_cashin', 'cat_cashout', 't_cf', 'nrt_cashin',
       'nrt_cashout', 't_inout', 'nr_p2pin', 'nr_p2pout', 'nr_saly',
       'ca_p2pin', 'ca_p2pout', 'ca_saly', 'ca_cashout', 'ca_cashin']

keyClient = data[data.is_good_saver==1]

In [23]:
similarClient= neiboursSearch.kneighbors(keyClient[walletCol],n_neighbors=200,return_distance=False) #[1].shape

fsimilarClient=similarClient.flatten()
unique  = list( set(fsimilarClient) )

In [24]:
data['is_similar_tobase'] = 0
data.loc[data.iloc[unique].index ,'is_similar_tobase'] =1
data.is_similar_tobase.fillna(0, inplace=True)
data['is_similar_tobase']=data.is_similar_tobase.astype(int)

data['all_suggestion'] = data.is_similar_tobase #| data.is_saly | data.indice

toTarget = data[data.all_suggestion==1]

toTarget = toTarget.drop(toTarget.query('cat_cashin==0 and cat_cashout==0').index)

removeMkajyUser= toTarget[ toTarget.usr_id.isin(approved.orange_id.unique()) ==False ] #[['usr_id','M1_Status']]

In [25]:
today = pd.datetime.today()
stoday = today.strftime('%d%M%y')

output = 'similartTo'+initialBase

In [93]:
removeMkajyUser[['usr_id']].to_csv(output+stoday+'.csv',index=False)

In [26]:
removeMkajyUser

,cat_cashin,cat_cashout,t_cf,nrt_cashin,nrt_cashout,t_inout,nr_p2pin,nr_p2pout,nr_saly,ca_p2pin,ca_p2pout,ca_saly,ca_cashout,ca_cashin,usr_id,cin,orange_id,is_good_saver,is_similar_tobase,all_suggestion
1,371300,400000,-28700,4,5,9,0,0,0,0,0,0,400000,371300,PT160509.1047.107626,715032021438,NaN,0,1,1
13,256000,268550,-12550,5,5,10,3,2,0,146000,121550,0,147000,110000,PT180903.1649.643233,411052005370,NaN,0,1,1
33,1756900,1582000,174900,15,14,29,2,1,0,31300,32000,0,1550000,1725600,PT180801.1516.287746,304012037554,NaN,0,1,1
35,1553000,1511850,41150,18,18,36,13,1,0,942200,41850,0,1470000,610800,PT110404.1457.000002,103322005978,NaN,0,1,1
65,718900,792150,-73250,20,19,39,2,17,0,14000,562150,0,230000,704900,PT170323.1515.689512,514032003726,NaN,0,1,1
71,318350,226150,92200,10,5,15,4,2,0,237800,31150,0,195000,80550,PT160907.1058.600145,301071033451,NaN,0,1,1
72,89000,80000,9000,3,2,5,0,0,0,0,0,0,80000,89000,PT170530.1515.486881,716011011546,NaN,0,1,1
90,1541750,1518075,23675,27,28,55,7,3,0,519800,156675,0,1361400,1021950,PT171007.1101.015491,101242144313,NaN,0,1,1
105,2000000,1955300,44700,3,8,11,0,6,0,0,1803300,0,152000,2000000,PT180802.1552.305983,18AC19125E3,NaN,0,1,1
118,2720300,3170000,-449700,9,14,23,2,0,0,639800,0,0,3170000,2080500,PT110323.0937.000158,718991031055,NaN,0,1,1


In [27]:
sept_recrut = pd.read_csv('recrutementSept_baseEpargne.csv', sep=',')

In [30]:
a = set(sept_recrut['usr_id'])

In [32]:
b = removeMkajyUser['usr_id']

In [35]:
len(a)

100636

In [36]:
len(b)

161654

In [38]:
r=set(b)-set(a)

In [39]:
len(r)

114403

In [42]:
new_list= pd.DataFrame(r,columns=['usr_id'])

In [44]:
new_list = new_list.merge(removeMkajyUser, on='usr_id', how='inner')

In [45]:
len(new_list)

114403

In [47]:
new_list[['usr_id']].to_csv(output+stoday+'.csv',index=False)

In [49]:
stoday

'110919'